<a href="https://colab.research.google.com/github/LKXoro/BEDU-Rstudio-postworks/blob/main/M3_Postworks5%2C6/Postwork_05_06_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Postworks 05 y 06: Limpieza de datos y agregaciones / Automatización y APIs

En esencia el Postwork 05 busca que limpiemos nuestro Dataset de valores nulos y probemos obtener información mediante agreaciones. Mientras que el Postwork 06 invita a automatizar el proceso de peticiones a una API. En esta entrega cambiaremos un poco el orden, primero realizaremos la automatización de la petición para obtener todos los datos y después los limpiaremos.

<p align="center">
<img src="https://empresas.blogthinkbig.com/wp-content/uploads/2019/03/janitor.png" height ="200" width = "200" alt=lego>
</p>

En postworks anteriores mencionamos que una petición a nuestra API de confianza, PandaScore, puede regresar como máximo 100 registros por petición y que las peticiones responden a parámetros de la URL. Creemos que la mejor forma de automatizar este proceso es con una función que nos devuelva la información que queremos, del juego que queremos de la API:


In [27]:
import requests #Importamos los paquete necesarios
import pandas as pd 
import math as mt
 
def PandaScore_get(juego,informacion): #Definimos nuestra función
    if isinstance(juego, str) and isinstance(informacion, str): #Nos aseguramos que los argumentos sean de tipo str
        registros = 100 #Establecemos el número de registros por página
        token = 'V3U1fnnWozJi3bbhZfrxdiAmN4-XWnX_4g6i_6iauwwh7w-V8Ns' #Token de acceso
        url = f"https://api.pandascore.co/{juego}/{informacion}?page[size]={registros}&token={token}" #URL con los parámetros deseados
        response = requests.get(url) #Realizamos la petición
        if response.status_code == 200: #Si todo salió bien continuamos
            Iteraciones = mt.ceil(float(response.headers['X-Total'])/registros) #Calculamos el número de páginas en base a todos los registros que hay en la API
            #Al dividir el número total de registros entre los registros solicitados obtenemos un aproximado, para asegurar que tomaremos todos, redondeamos al entero más alto
            for i in range(1,Iteraciones+1): #Ciclo for para obtener todos los registros de cada página
                url = f"https://api.pandascore.co/{juego}/{informacion}?page[size]={registros}&page[number]={i}&token=V3U1fnnWozJi3bbhZfrxdiAmN4-XWnX_4g6i_6iauwwh7w-V8Ns"
                #URL el parámetro de página deseada
                response = requests.get(url) #Realizamos la petición correspondiente
                data = response.json() #Transformamos la respuesta 
                if i == 1: #En la primer iteración se crea un Dataframe
                    df = pd.DataFrame.from_dict(data)
                else: #En el resto de iteraciones se van añadiendo los nuevos registros
                    df = df.append(pd.DataFrame.from_dict(data),ignore_index=True) 
            return df # Finalmente se devuelve el Dataframe con todos los registros
        else: #Si la petición no se ejecutó correctamente mandamos un mensaje con el error devuelto
            return 'Error de petición: '+str(response.status_code)  
    else: #Si los argumentos no son de tipo str regresamos un mensaje
        return 'Por favor coloque un argumento válido en formato string.'
  

Veamos a la función en acción:

In [31]:
PandaScore_get('lol',1)

'Por favor coloque un argumento válido en formato string.'

In [32]:
PandaScore_get('lol','matchs')

'Error de petición: 404'

Si los argumentos son correctos se nos devolverá el Dataframe correspondiente, el tiempo de ejecución varia dependiendo de la información solicitada, en el caso de los enfrentamientos tenemos 18694 registros, por lo que la función tomará un par de minutos al solo poder realizar 100 peticiones, en general consumiriamos 187 de nuestros 1000 intentos por hora:

In [ ]:
df_matches = PandaScore_get('lol','matches')

Observemos el Dataframe que se nos devuelve:

In [42]:
df_matches

,forfeit,number_of_games,name,slug,videogame,begin_at,tournament,streams,results,winner_id,modified_at,league,official_stream_url,draw,winner,games,league_id,live_embed_url,videogame_version,status,game_advantage,serie_id,tournament_id,scheduled_at,id,serie,match_type,end_at,original_scheduled_at,detailed_stats,rescheduled,opponents,live
0,False,5,Round 3: TBD vs TBD,2021-03-11-c3b8c131-fac1-4b71-b339-f1d0387914a6,"{'id': 1, 'name': 'LoL', 'slug': 'league-of-le...",2021-03-11T17:00:00Z,"{'begin_at': '2021-03-09T17:00:00Z', 'end_at':...","{'english': {'embed_url': None, 'raw_url': Non...",[],NaN,2021-03-07T20:20:32Z,"{'id': 4302, 'image_url': 'https://cdn.pandasc...",None,False,None,"[{'begin_at': None, 'complete': False, 'detail...",4302,None,None,not_started,None,3254,5649,2021-03-11T17:00:00Z,586007,"{'begin_at': '2021-01-19T17:00:00Z', 'descript...",best_of,None,2021-03-11T17:00:00Z,True,False,[],"{'opens_at': None, 'supported': False, 'url': ..."
1,False,3,Round 2 Match 2: MOUZ vs BIG,mousesports-vs-berlin-international-gaming-202...,"{'id': 1, 'name': 'LoL', 'slug': 'league-of-le...",2021-03-10T20:00:00Z,"{'begin_at': '2021-03-09T17:00:00Z', 'end_at':...","{'english': {'embed_url': None, 'raw_url': Non...","[{'score': 0, 'team_id': 16}, {'score': 0, 'te...",NaN,2021-03-07T20:19:53Z,"{'id': 4302, 'image_url': 'https://cdn.pandasc...",None,False,None,"[{'begin_at': None, 'complete': False, 'detail...",4302,None,None,not_started,None,3254,5649,2021-03-10T20:00:00Z,586006,"{'begin_at': '2021-01-19T17:00:00Z', 'descript...",best_of,None,2021-03-10T20:00:00Z,True,False,"[{'opponent': {'acronym': 'MOUZ', 'id': 16, 'i...","{'opens_at': None, 'supported': False, 'url': ..."
2,False,3,Round 2 Match 1: TBD vs TBD,2021-03-10-7412caf0-3f45-40e1-b763-80d005a04162,"{'id': 1, 'name': 'LoL', 'slug': 'league-of-le...",2021-03-10T17:00:00Z,"{'begin_at': '2021-03-09T17:00:00Z', 'end_at':...","{'english': {'embed_url': None, 'raw_url': Non...",[],NaN,2021-03-07T20:19:33Z,"{'id': 4302, 'image_url': 'https://cdn.pandasc...",None,False,None,"[{'begin_at': None, 'complete': False, 'detail...",4302,None,None,not_started,None,3254,5649,2021-03-10T17:00:00Z,586005,"{'begin_at': '2021-01-19T17:00:00Z', 'descript...",best_of,None,2021-03-10T17:00:00Z,True,False,[],"{'opens_at': None, 'supported': False, 'url': ..."
3,False,3,Round 1 Match 2: SKP vs S04.E,sk-gaming-prime-vs-fc-schalke-04-evolution-202...,"{'id': 1, 'name': 'LoL', 'slug': 'league-of-le...",2021-03-09T20:00:00Z,"{'begin_at': '2021-03-09T17:00:00Z', 'end_at':...","{'english': {'embed_url': None, 'raw_url': Non...","[{'score': 0, 'team_id': 247}, {'score': 0, 't...",NaN,2021-03-07T20:18:12Z,"{'id': 4302, 'image_url': 'https://cdn.pandasc...",None,False,None,"[{'begin_at': None, 'complete': False, 'detail...",4302,None,None,not_started,None,3254,5649,2021-03-09T20:00:00Z,586004,"{'begin_at': '2021-01-19T17:00:00Z', 'descript...",best_of,None,2021-03-09T20:00:00Z,True,False,"[{'opponent': {'acronym': 'SKP', 'id': 247, 'i...","{'opens_at': None, 'supported': False, 'url': ..."
4,False,3,Round 1 Match 1: GL vs USE,team-gamerlegion-vs-unicorns-of-love-sexy-edit...,"{'id': 1, 'name': 'LoL', 'slug': 'league-of-le...",2021-03-09T17:00:00Z,"{'begin_at': '2021-03-09T17:00:00Z', 'end_at':...","{'english': {'embed_url': None, 'raw_url': Non...","[{'score': 0, 'team_id': 126830}, {'score': 0,...",NaN,2021-03-07T20:17:44Z,"{'id': 4302, 'image_url': 'https://cdn.pandasc...",None,False,None,"[{'begin_at': None, 'complete': False, 'detail...",4302,None,None,not_started,None,3254,5649,2021-03-09T17:00:00Z,586003,"{'begin_at': '2021-01-19T17:00:00Z', 'descript...",best_of,None,2021-03-09T17:00:00Z,True,False,"[{'opponent': {'acronym': 'GL', 'id': 126830, ...","{'opens_at': None, 'supported': False, 'url': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18689,False,1,DIG vs C9,dignitas-vs-cloud9-2015-05-31,"{'id': 1, 

Veamos las columnas y tipos de datos:

In [34]:
df_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18694 entries, 0 to 18693
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   forfeit                18694 non-null  bool   
 1   number_of_games        18694 non-null  int64  
 2   name                   18694 non-null  object 
 3   slug                   18694 non-null  object 
 4   videogame              18694 non-null  object 
 5   begin_at               18662 non-null  object 
 6   tournament             18694 non-null  object 
 7   streams                18694 non-null  object 
 8   results                18694 non-null  object 
 9   winner_id              17758 non-null  float64
 10  modified_at            18694 non-null  object 
 11  league                 18694 non-null  object 
 12  official_stream_url    10896 non-null  object 
 13  draw                   18694 non-null  bool   
 14  winner                 17758 non-null  object 
 15  ga

Como podemos ver tenemos 33 columnas, un número considerable, observemos que contienen algunas y si nos son de utilidad. Empecemos por los campos que contienen listas:

In [44]:
df_matches['videogame'][0] 

{'id': 1, 'name': 'LoL', 'slug': 'league-of-legends'}

El campo _videogame_ puede que haya sido de utilidad para la API al momento de regresarnos una respuesta, sin embargo, es  redundante para nosotros.

In [48]:
df_matches['streams'][0]

{'english': {'embed_url': None, 'raw_url': None},
 'official': {'embed_url': None, 'raw_url': None},
 'russian': {'embed_url': None, 'raw_url': None}}

In [47]:
df_matches['streams'][300]

{'english': {'embed_url': None, 'raw_url': None},
 'official': {'embed_url': 'https://player.twitch.tv/?channel=riotgamesru',
  'raw_url': 'https://www.twitch.tv/riotgamesru'},
 'russian': {'embed_url': 'https://player.twitch.tv/?channel=riotgamesru',
  'raw_url': 'https://www.twitch.tv/riotgamesru'}}

In [57]:
df_matches.loc[300,'official_stream_url']

'https://www.twitch.tv/riotgamesru'

El campo _streams_ nos regresa las direcciones URL de las transmisiones en vivo en inglés y ruso, mientras que el campo _official stream url_ nos regresa únicamente la URL del stream oficial. Creemos es mejor conservar este último campo con la dirección oficial a un campo con una lista  más difícil de manejar.

In [60]:
df_matches['tournament'][0]

{'begin_at': '2021-03-09T17:00:00Z',
 'end_at': None,
 'id': 5649,
 'league_id': 4302,
 'live_supported': False,
 'modified_at': '2021-03-08T14:50:51Z',
 'name': 'Playoffs',
 'prizepool': None,
 'serie_id': 3254,
 'slug': 'league-of-legends-prime-league-pro-division-spring-2021-playoffs',
 'winner_id': None,
 'winner_type': 'Team'}

El campo _tournament_ nos devuelve el torneo correspondiente. Sin embargo, contamos con el _tournament id_, y podemos realizar peticiones de torneos si lo deseamos, por lo que de ser necesario podriamos realizar un **join** y observar a que torneo corresponde cada enfrentamiento. 

In [65]:
df_matches['modified_at'].head() 

0    2021-03-07T20:20:32Z
1    2021-03-07T20:19:53Z
2    2021-03-07T20:19:33Z
3    2021-03-07T20:18:12Z
4    2021-03-07T20:17:44Z
Name: modified_at, dtype: object

La última fecha de modificación respecto a la API no nos resulta útil del todo a nosotros.

In [66]:
df_matches['league'][0]

{'id': 4302,
 'image_url': 'https://cdn.pandascore.co/images/league/image/4302/220px-Prime_Leaguelogo_square.png',
 'modified_at': '2020-01-20T16:20:40Z',
 'name': 'Prime League Pro Division',
 'slug': 'league-of-legends-prime-league-pro-division',
 'url': 'https://www.primeleague.gg/de/start'}

Mismo caso que con los torneos, si convservamos el _league id_ no deberiamos tener problemas.

In [70]:
df_matches['games'][300]

[{'begin_at': None,
  'complete': False,
  'detailed_stats': True,
  'end_at': None,
  'finished': False,
  'forfeit': False,
  'id': 222685,
  'length': None,
  'match_id': 583283,
  'position': 1,
  'status': 'not_started',
  'video_url': None,
  'winner': {'id': None, 'type': None},
  'winner_type': None}]

El campo _games_ debería entregar información de cada partida en el enfrentamiento, pero en general esos datos podemos encontrarlos en otros campos que ya tenemos.

In [74]:
df_matches.loc[300,'videogame_version']